# README
* 呼叫 `get_AML_person(content, ckip_name, mode=0, binary=0)` 即可
    * `mode=0`, `binary` 無作用；使用模型預測 binary
    * `mode=1`, `binary` 用來放其他模型的 binary 分類輸出 (int 1 or 0)

In [1]:
import pandas as pd
import ast
import numpy as np
import re
from zhon.hanzi import stops, non_stops
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertConfig, BertTokenizer, BertModel

import datetime #####
import math

PRETRAINED_MODEL_NAME = 'bert_wwm_pretrain_tbrain' # pretrained_bert_wwm
MODEL_PATH = './model_final_state_dict/pre_bert_wwm_bio_only_EPOCHES_19.pkl'

def clean_string(content):
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
    result = []
    for i in range(math.ceil(len(content) / 511)):
        result.append(content[i*512 : i*512+511])
    return result

def bio_2_string(have_AML, BIO_tagging, ckip_result, origin_text, BIO_prob):
  result = []
  if (have_AML.item() == 0):
    result.append('')
  else:
#     for j in range(1, 512):
#       if (BIO_tagging[j] == 0):
#         start = j
#         end = j + 1
#         while (end < 512 and BIO_tagging[end] == 1):
#           end += 1
#         if (end > start + 1):
#           if (start <= 3):
#               s = origin_text[start-1 : end + 2] # -1 for CLS
# #               print(BIO_prob[start : end + 3])
#           else:
#               s = origin_text[start-1-1 : end + 2] # -1 for CLS
# #               print(BIO_prob[start-1 : end + 3])
# #           print('origin_span: ', origin_text[start-1 : end-1])
# #           print(s)
#           for k in range(len(ckip_result)):
#             if (len(ckip_result[k]) < 2):
#               continue
#             elif (re.findall(r"[%s]+" %non_stops, ckip_result[k]) != [] \
#                      or re.findall(r"[%s]+" %stops, ckip_result[k]) != []): # 有標點
#               continue
#             found = s.find(ckip_result[k])
#             if (found != -1):
# #               print('found: ', found)
#               result.append(ckip_result[k])
    full_str = ""
    for j in range(1, 512):
      if (BIO_tagging[j] == 0):
        start = j
        end = j + 1
        while (end < 512 and BIO_tagging[end] == 1):
          end += 1
        if (end > start + 1):
          if (start <= 3):
              s = origin_text[start-1 : end + 2] # -1 for CLS
#               print(BIO_prob[start : end + 3])
          else:
              s = origin_text[start-1-1 : end + 2] # -1 for CLS
#               print(BIO_prob[start-1 : end + 3])
#           print('origin_span: ', origin_text[start-1 : end-1])
#           print(s)
          full_str += s
    for k in range(len(ckip_result)):
      if (len(ckip_result[k]) < 2):
        continue
      elif (re.findall(r"[%s]+" %non_stops, ckip_result[k]) != [] \
                 or re.findall(r"[%s]+" %stops, ckip_result[k]) != []): # 有標點
        continue
      found = full_str.find(ckip_result[k])
      if (found != -1):
#       print('found: ', found)
        result.append(ckip_result[k])
      
    if (len(result) == 0):
      result.append('')
  return result

def get_predictions(model, tokens_tensors, segments_tensors, masks_tensors, ckip_names, origin_text, mode, binary):
  result = []
  with torch.no_grad():
      tokens_tensors = tokens_tensors.to("cuda:0")
      segments_tensors = segments_tensors.to("cuda:0")
      masks_tensors = masks_tensors.to("cuda:0")
      start = datetime.datetime.now().timestamp() ######
      outputs = model(input_ids=tokens_tensors, 
                  token_type_ids=segments_tensors, 
                  attention_mask=masks_tensors)
      end = datetime.datetime.now().timestamp()###########
      print("through model time: ", end-start) ##########
      
      count = outputs[0].shape[0]
      for i in range(count):  # run batchsize times
        if (mode == 0):
            have_AML = outputs[0][i].argmax()
        else:
            have_AML = torch.tensor([binary])
        BIO_pred = outputs[0][i].argmax(1) # 3*512 into class label
        ckip_names_list = ast.literal_eval(ckip_names) # string to list
#         print(origin_text[i])
        start = datetime.datetime.now().timestamp() ######
        r = bio_2_string(have_AML, BIO_pred, ckip_names_list, origin_text[i], outputs[0][i])  #####
        end = datetime.datetime.now().timestamp()###########
        print("bio_2_string time: ", end-start) ##########
        result.append(r)
  return result

""" model budling """
class AMLPredictModel(nn.Module):
    def __init__(self, config):
        super(AMLPredictModel, self).__init__()
        self.bert = BertModel.from_pretrained(PRETRAINED_MODEL_NAME, config = config)
        self.BIO_classifier = nn.Sequential(
                        nn.Linear(config.hidden_size, 3),
        ) # BIO tagging
        self.softmax = nn.Softmax(-1)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        BIO = self.BIO_classifier(outputs[0]) # 512*HIDDENSIZE word vectors
        BIO = self.softmax(BIO)
        
        outputs = (BIO,) + outputs[2:]
        return outputs
    
config = BertConfig.from_pretrained(PRETRAINED_MODEL_NAME, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
model = AMLPredictModel(config)
model.load_state_dict(torch.load(MODEL_PATH))
    
def get_AML_person(model, content, ckip_name, mode=0, binary=0):
    start = datetime.datetime.now().timestamp() ######
    content = clean_string(content)
    end = datetime.datetime.now().timestamp()###########
    print("clean_string time: ", end-start) ##########
    start = datetime.datetime.now().timestamp() #####
    test_input_dict = tokenizer.batch_encode_plus(content, 
                          add_special_tokens=True,
                          max_length=512,
                          return_special_tokens_mask=True,
                          pad_to_max_length=True,
                          return_tensors='pt',
                          truncation=True)
    end = datetime.datetime.now().timestamp()###########
    print("tokenizer time: ", end-start) ##########


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model = model.eval()
    r = get_predictions(model, test_input_dict['input_ids'], test_input_dict['token_type_ids'], test_input_dict['attention_mask'],\
                           ckip_name, content, mode, binary)
#     print(result)
    result = set()
    for i in range(len(r)):
        result = result | set(r[i])
    return result

In [2]:
df = pd.read_csv('./dataset/2020-07-28.csv')
# df

In [3]:
test = 157
print(df.loc[test, 'article'])
print(df.loc[test, 'ckip_name'])
print(df.loc[test, 'predict_name'])

陳致遠電玩弊案涉案官警人數創歷史之最，前北巿警局督察員張惠聖更七審判刑5年6月，最高法院日前駁回上訴定讞。全案纏訟近24年，張惠聖成為本案最晚判決確定的起訴被告。本案一審判張惠聖13年，二審也判13年，案經最高法院發回六次，高院更一至更六審判他10年6月至5年6月不等刑期，更七審則判刑5年6月。陳致遠案在民國85年4月間爆發後，震驚社會，捲入官警人數創下歷史之最，讓多名警界「明日之星」中箭落馬。檢察官當年針對此案共起訴7波、194名被告，包括官警38人。陳致遠遭判1年3月又15日徒刑定讞，已服刑完畢。台灣高等法院更七審指出，陳致遠為圖不法暴利，進口電動玩具及零件，經裝配及維修後，以「金」字招牌經營賭博性電動遊樂場、遊藝場及電動玩具店，陸續在台北市中山區等地開店。陳致遠為免賭博性電玩店被警方查報、列管及取締，指示陳詩惟處理行賄中山區員警的公關工作，陳詩惟於民國81年底經由時任台北市警察局士林分局刑事組小隊長許原翰（通緝中）介紹認識時任台北市警察局督察室督察員張惠聖。張惠聖從81年6月間起至82年2月間止，負責中山分局、士林分局員警內外勤業務的督導考核，負有受理調查民眾檢舉賭博性電玩店涉及員警的風紀案件，卻違背職務收受賄賂新台幣20萬元。張惠聖後來負責大安、南港分局，便協助陳致遠及陳詩惟行賄送錢給後來接任負責中山區的督察員陳博梅（判決7年定讞）110萬元、謝明彬（通緝中）90萬元。更七審指出，張惠聖收受陳致遠集團賄賂，又擔任「白手套」行賄其他員警，考量他偵查中自白犯行，未自動繳交犯罪所得，依違背職務收賄罪、交付賄賂罪共2罪論罪，再依刑事妥速審判法減刑，判他應執行5年6月，案經最高法院駁回而確定。
['更七審', '陳博梅', '許原翰', '謝明彬', '陳致遠', '張惠聖', '陳詩惟']
['陳致遠', '張惠聖', '許原翰', '陳詩惟']


In [4]:
t = df.loc[test, 'article']
ckip_n = df.loc[test, 'ckip_name']

In [10]:
t = '台北地檢署偵辦台北市警中山分局中山一派出所集體貪瀆弊案，上月初判中山一派出所前、後3任所長林子芸、楊修白、劉靜怡、林子芸妻子林思賢、行賄「白手套」黃念心5人無罪，台北地檢署不服判決，對5人提起上訴。  檢調偵辦台北市中山區「立邦」酒店媒介外籍女子賣淫案，查出林子芸等官警自2004年起至2017年按月收業者4萬元賄款，以不臨檢、通風報信方式包庇酒店經營；「夜王」酒店從2007年起到2017年每月向派出所員警行賄1萬5000元，將涉案員警、業者多人起訴。  台北地院審理，上月6日將員警李石良判刑16年、判游怡如13年、判紀宏白8年、判張佳雯14年、判紀炳場14年、判陳宏洲10年10月、判莊琦良12年半，另員警曾學函、楊惠志、蔣盈君獲判緩刑，不過，林子芸、楊修白、劉靜怡、林思賢、黃念心5人獲判無罪。  台北地檢署檢察官收判後，認為法官判決林子芸等5人無罪的理由違反經驗法則、論理法則，日前向台灣高等法院提起上訴'
ckip_n = "['莊琦良', '黃念心', '判紀宏白', '張佳雯', '楊修白', '林思賢', '判紀炳場', '林子芸', '曾學函', '劉靜怡', '蔣盈君', '楊惠志', '陳宏洲', '游怡如', '李石良']"

In [8]:
len(t)

407

In [11]:
import math

start = datetime.datetime.now().timestamp()
ans = get_AML_person(model, t, ckip_n, mode=1, binary=1)
end = datetime.datetime.now().timestamp()
print('ans: ', ans)
print('total time: ', end-start)

clean_string time:  2.9087066650390625e-05
tokenizer time:  0.0017168521881103516
through model time:  0.006949901580810547
bio_2_string time:  0.02967095375061035
ans:  {'張佳雯', '黃念心', '曾學函', '林思賢', '林子芸', '判紀炳場', '劉靜怡', '楊惠志', '游怡如', '李石良', '陳宏洲', '蔣盈君', '莊琦良', '楊修白', '判紀宏白'}
total time:  0.057389020919799805


In [7]:
a = {'張惠聖', '陳詩惟', '許原翰', '更七審', '陳致遠'}
b = {'謝明彬', '張惠聖', '陳詩惟', '陳博梅', '陳致遠'}

a | b

{'張惠聖', '更七審', '許原翰', '謝明彬', '陳博梅', '陳致遠', '陳詩惟'}

In [9]:
s = "asdf"
s+="wtr"
s

'asdfwtr'